Credits to the owner of this github https://github.com/ultralytics/ultralytics/blob/main/ultralytics/cfg/datasets/VOC.yaml for providing us the required dataset for this project

In [5]:
pip install ultralytics

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [4]:
import xml.etree.ElementTree as ET
from pathlib import Path
from ultralytics.utils.downloads import download
from ultralytics.utils import ASSETS_URL, TQDM

yaml = {
    "path": "VOC",
    "names": {
        0: "aeroplane",
        1: "bicycle",
        2: "bird",
        3: "boat",
        4: "bottle",
        5: "bus", 
        6: "car", 
        7: "cat", 
        8: "chair", 
        9: "cow",
        10: "diningtable", 
        11: "dog", 
        12: "horse", 
        13: "motorbike",
        14: "person", 
        15: "pottedplant", 
        16: "sheep", 
        17: "sofa",
        18: "train", 
        19: "tvmonitor"
    }
}

def convert_label(path, lb_path, year, image_id):
    def convert_box(size, box):
        dw, dh = 1.0/size[0], 1.0/size[1]
        x, y, w, h = (box[0] + box[1]) / 2.0 - 1, (box[2] + box[3]) / 2.0 - 1, box[1] - box[0], box[3] - box[2]
        return x * dw, y * dh, w * dw, h * dh
    
    with open(path /f"VOC{year}/Annotations/{image_id}.xml", encoding="utf-8") as in_file, \
        open(lb_path, "w", encoding="utf-8") as out_file:
        try:
            tree = ET.parse(in_file)
        except ET.ParseError as e:
            print(f"Error parsing XML for {image_id}: {e}")
            return
        
        root = tree.getroot()
        size = root.find("size")
        if size is None:
            return
        
        w = int(size.find("width").text)
        h = int(size.find("height").text)

        names = list(yaml["names"].values())  # names list
        for obj in root.iter("object"):
            cls = obj.find("name").text
            # Check for 'difficult' attribute gracefully
            difficult_elem = obj.find("difficult")
            is_difficult = int(difficult_elem.text) != 1 if difficult_elem is not None else 0

            if cls in names and not is_difficult:
                xmlbox = obj.find("bndbox")
                bb = convert_box((w, h), [float(xmlbox.find(x).text) for x in ("xmin", "xmax", "ymin", "ymax")])
                cls_id = names.index(cls)  # class id
                out_file.write(" ".join(str(a) for a in (cls_id, *bb)) + "\n")


print("Starting PASCAL VOC Dataset Download and Conversion...")
dir = Path(yaml["path"])  # dataset root dir (will be 'VOC')
urls = [
    f"{ASSETS_URL}/VOCtrainval_06-Nov-2007.zip",  # 446MB
    f"{ASSETS_URL}/VOCtest_06-Nov-2007.zip",  # 438MB
    f"{ASSETS_URL}/VOCtrainval_11-May-2012.zip",  # 1.95GB
]


print("1. Downloading ZIP archives (approx. 2.8 GB total)...")
download(urls, dir=dir / "images", threads=3, exist_ok=True)


print("2. Converting annotations and reorganizing files...")
path = dir / "images/VOCdevkit"
for year, image_set in ("2012", "train"), ("2012", "val"), ("2007", "train"), ("2007", "val"), ("2007", "test"):
    imgs_path = dir / "images" / f"{image_set}{year}"
    lbs_path = dir / "labels" / f"{image_set}{year}"
    imgs_path.mkdir(exist_ok=True, parents=True)
    lbs_path.mkdir(exist_ok=True, parents=True)

    try:
        with open(path / f"VOC{year}/ImageSets/Main/{image_set}.txt") as f:
            image_ids = f.read().strip().split()
    except FileNotFoundError:
        print(f"Warning: Image set file not found for {image_set}{year}. Skipping conversion.")
        continue

    for id in TQDM(image_ids, desc=f"{image_set}{year}"):
        f = path / f"VOC{year}/JPEGImages/{id}.jpg"  # old img path
        lb_path = (lbs_path / f.name).with_suffix(".txt")  # new label path
        
        # Check if the image file exists before trying to rename/move it
        if f.exists():
            f.replace(imgs_path / f.name)  # move image
            convert_label(path, lb_path, year, id)  # convert labels to YOLO format
        else:
             # This is a common issue with VOC dataset where ImageSets reference missing files
             print(f"Warning: Image file not found for ID {id}. Skipping.")

print("\n🎉 PASCAL VOC dataset download and conversion complete!")
print(f"The dataset is located in the '{dir}' directory.")

Starting PASCAL VOC Dataset Download and Conversion...
1. Downloading ZIP archives (approx. 2.8 GB total)...
Unzipping VOC\images\VOCtest_06-Nov-2007.zip to D:\Project\PASCAL VOI\VOC\images\VOCdevkit...: 100% ━━━━━━━━━━━━ 10357/10357 159.8files/s 1:05:05<1.8s5
Unzipping VOC\images\VOCtrainval_06-Nov-2007.zip to D:\Project\PASCAL VOI\VOC\images\VOCdevkit...: 100% ━━━━━━━━━━━━ 10945/10945 165.6files/s 1:06<0.0s
Unzipping VOC\images\VOCtrainval_11-May-2012.zip to D:\Project\PASCAL VOI\VOC\images\VOCdevkit...: 100% ━━━━━━━━━━━━ 40189/40189 354.6files/s 1:53<0.1s
2. Converting annotations and reorganizing files...
train2012: 100% ━━━━━━━━━━━━ 5717/5717 73.5it/s 1:18<0.1ss
val2012: 100% ━━━━━━━━━━━━ 5823/5823 89.8it/s 1:05<0.1sss
train2007: 100% ━━━━━━━━━━━━ 2501/2501 81.0it/s 30.9s<0.0s
val2007: 100% ━━━━━━━━━━━━ 2510/2510 76.1it/s 33.0s<0.1s
test2007: 100% ━━━━━━━━━━━━ 4952/4952 78.5it/s 1:03<0.1ss

🎉 PASCAL VOC dataset download and conversion complete!
The dataset is located in the 'VOC' 

In [1]:
import xml.etree.ElementTree as ET
from pathlib import Path
from ultralytics.utils.downloads import download
from ultralytics.utils import ASSETS_URL, TQDM
import json

DATASET_ROOT = Path("VOC")

In [2]:
def xml_to_json(xml_path, json_path, img_id):
    try:
        tree = ET.parse(xml_path)
    except ET.ParseError as e:
        print(f"XML parse error for {img_id}: {e}")
        return
    
    root = tree.getroot()

    size = root.find("size")
    if size is None:
        return
    
    annotation = {
        "image_id" : img_id,
        "size" : {
            "width" : int(size.find("width").text),
            "height" : int(size.find("height").text),
        },
        "objects": []
    }

    for obj in root.findall("object"):
        name = obj.find("name").text

        diff_element = obj.find("difficult")
        diff = int(diff_element.text) if diff_element is not None else 0

        bbox = obj.find("bndbox")
        annotation["objects"].append({
            "class" : name,
            "bbox"  : {
                "xmin": int(bbox.find("xmin").text),
                "ymin": int(bbox.find("ymin").text),
                "xmax": int(bbox.find("xmax").text),
                "ymax": int(bbox.find("ymax").text),
            },
            "difficult": diff
        })

    with open(json_path, "w", encoding="utf-8") as f:
        json.dump(annotation, f, indent=2)

In [3]:
print("Downloading Datasets....")

urls = [
    f"{ASSETS_URL}/VOCtrainval_11-May-2012.zip",  # 1.95GB
]

download(urls, dir=DATASET_ROOT/"images", threads=3, exist_ok=True)
print("Download complete")

Unzipping VOC\images\VOCtrainval_11-May-2012.zip to D:\Project\PASCAL VOI\VOC\images\VOCdevkit...: 100% ━━━━━━━━━━━━ 40189/40189 707.2files/s 56.8s<0.1s
Download complete


In [6]:
print("Converting annotations...")

voc_root = DATASET_ROOT/"images/VOCdevkit"

for year, img_set in (
    ("2012", "train"),
    ("2012", "val"),
):
    json_dir = DATASET_ROOT/"annotations_json"/f"{img_set}{year}"
    json_dir.mkdir(parents=True, exist_ok=True)

    try:
        with open(voc_root/f"VOC{year}/ImageSets/Main/{img_set}.txt") as f:
            img_ids = f.read().strip().split()
    except FileNotFoundError:
        print(f"Skipping {img_set}{year}")
        continue
    for img_id in TQDM(img_ids, desc=f"{img_set}"):
        xml_path = voc_root/f"VOC{year}/Annotations/{img_id}.xml"
        json_path = json_dir/f"{img_id}.json"

        if xml_path.exists():
            xml_to_json(xml_path, json_path, img_id)
        else:
            print(f"Missing XML: {img_id}, skipping.")
print("\n🎉 VOC XML → JSON conversion finished successfully!")

Converting annotations...
train: 100% ━━━━━━━━━━━━ 5717/5717 89.1it/s 1:04<0.1ss
val: 100% ━━━━━━━━━━━━ 5823/5823 89.9it/s 1:05<0.1ss

🎉 VOC XML → JSON conversion finished successfully!
